RAG chatbot 
Version 1.0 

Import Libraries

In [2]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.retrievers import MultiQueryRetriever
import chromadb

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Suppress ChromaDB telemetry warnings as they are useless as ChromaDB trying to send usage data but failing because it can't find its own version information
import logging
logging.getLogger("chromadb").setLevel(logging.ERROR)

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

LOAD PDF

In [ ]:
# Load PDF
local_path = "TaskFlow.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path, languages=["deu"])
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload documents")

#TODO: language detection!!!!


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


PDF loaded successfully: TaskFlow.pdf


SPLIT TEXT INTO CHUNKS

In [4]:
#Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Number of text chunks: {len(chunks)}")

Number of text chunks: 6


CREATE VECTOR DATABASE

In [5]:
# Create vector database
client = chromadb.PersistentClient(path="./chroma_data")
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="rag_chatbot_collection",
    client=client
)
print("Vector database created successfully.")


Vector database created successfully.


Set up LLM and Retrieval

In [6]:
#Set up LLM and retrieval
local_model = "llama3.2"
llm = ChatOllama(model=local_model, temperature=0.1)

# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a query rephrasing assistant for a vector database search.
    Your task is to generate **3** alternative, semantically diverse versions of the user's question. These variations should explore different terminology, structures, or angles of the original query to maximize relevant document retrieval.
    Provide only the rephrased questions, with each one separated by a newline. Do not include any introductory phrases, explanations, or the original question itself.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

Create chain

In [ ]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


# Create chain (pipeline)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Chat with PDF

In [8]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [9]:
# Example 1
chat_with_pdf("What is the main idea of this document?")

The main idea of this document appears to be a technical specification for a task management system, outlining requirements and guidelines for its development, implementation, and functionality. The document covers various aspects such as performance, reliability, user experience, and data persistence, providing a detailed overview of the expected features and functionalities of the system.

In [10]:
# Example 2
chat_with_pdf("What are the two options for technical implementation of this system?")

Die beiden Optionen für die technische Implementierung des Systems sind:

1. Implementierung als Webanwendung (Spring o.ä.)
2. Implementierung als Desktop-Anwendung (Swing)

In [11]:
# Example 4
chat_with_pdf("What should I do? Can you give me some steps?")

Basierend auf dem bereitgestellten Kontext, die folgenden Schritte empfehle ich:

1. **Verstehe die Anforderungen**: Lesen Sie den Dokumententext sorgfältig und verstehen Sie die Anforderungen an das Task-Management-System.

2. **Entwickle eine Datenbank**: Implementieren Sie eine SQLite-Datenbank, um alle Aufgaben zu speichern. Stellen Sie sicher, dass das System beim Start automatisch die gespeicherten Daten lädt und bei jeder Änderung die Daten aktualisiert.

3. **Implementiere die REST-API**: Entwickeln Sie eine REST-API mit den folgenden Endpunkten:
 - GET /api/tasks: Alle Aufgaben abrufen
 - GET /api/tasks/{id}: Einzelne Aufgabe abrufen
 - POST /api/tasks: Neue Aufgabe erstellen
 - PUT /api/tasks/{id}: Aufgabe aktualisieren
 - DELETE /api/tasks/{id}: Aufgabe löschen
 - GET /api/tasks/stats: Statistiken abrufen

4. **Implementiere die Aufgabenverwaltung**: Entwickeln Sie eine Funktion zur Erstellung, Bearbeitung und Löschung von Aufgaben. Stellen Sie sicher, dass Benutzer eine Bestätigung erhalten müssen, bevor eine Aufgabe gelöscht wird.

5. **Implementiere die Filterung und Sortierung**: Implementieren Sie eine Filter- und Sortierfunktion für die Aufgabenliste, die nach Priorität und Fälligkeitsdatum sortiert ist.

6. **Implementiere die Statistikübersicht**: Entwickeln Sie eine Funktion zur Bereitstellung einer Statistikübersicht, die die Anzahl der Aufgaben pro Status und die Anzahl überfälliger Aufgaben zeigt.

7. **Optimieren für Performance**: Stellen Sie sicher, dass das System mindestens 100.000 Aufgaben ohne merkliche Verzögerung verwalten kann. Die Startzeit der Anwendung sollte innerhalb von 2 Sekunden liegen und Filteroperationen sollten innerhalb von 100ms Ergebnisse liefern.

8. **Implementiere die Zuverlässigkeit**: Implementieren Sie Robustheit gegenüber fehlerhaften Eingaben und stellen Sie sicher, dass das System bei ungültigen Befehlen nicht abstürzt.

9. **Implementiere die Wartbarkeit**: Modifizieren Sie den Code so, dass er modular aufgebaut ist mit klarer Trennung zwischen den implementierten Schichten (Präsentation, Daten, Logik ...). Stellen Sie sicher, dass eine Testabdeckung von mindestens 80% für die Geschäftslogik besteht.

10. **Implementiere Variante 1**: Wenn zwei Varianten zur Verfügung stehen, wählen Sie Variante 1 und implementieren Sie sie als Webanwendung (Spring o.ä.).